# Finnhub API
Recuperando US stock data de API financiera.

## Readme
Para correr este notebook es necesario un API Key de Finnhub. Puede ser configurado en un archivo
`.env` bajo la carpeta `api-etl`, con nombre de variable `FINNHUB_API_TOKEN`.
También es posible cargarla como variable de entorno del sistema opertivo.

## Imports

In [1]:
import requests
import os
import pandas as pd

## Load ENVs

In [2]:
%load_ext dotenv
%dotenv

## API Class

In [3]:
# La API tiene una libreria de Python, pero vamos a armar nuestra propia versión (más acorde a la consigna).

class Finnhub:
  BASE_URL="https://finnhub.io/api/v1"

  def __init__(self, token):
    self.token = token
  
  def quote(self, symbol):
    params = { "symbol": symbol, "token": self.token }
    res = requests.get(f'{self.BASE_URL}/quote', params=params)
    res.raise_for_status()
    return res.json()

Probando la API con el simbolo de Apple

In [4]:
api = Finnhub(token=os.environ['FINNHUB_API_TOKEN'])
api.quote("AAPL")

{'c': 172.035,
 'd': -1.275,
 'dp': -0.7357,
 'h': 172.23,
 'l': 170.51,
 'o': 171.7,
 'pc': 173.31,
 't': 1711654682}

## Extraer y Guardar

Definimos nuestras acciones deseadas

In [5]:
stocks = [
  {'name': 'Apple', 'symbol': 'AAPL'},
  {'name': 'Google', 'symbol': 'GOOGL'},
  {'name': 'Coca Cola', 'symbol': 'KO'},
  {'name': 'Microsoft', 'symbol': 'MSFT'},
  {'name': 'Amazon', 'symbol': 'AMZN'},
  {'name': 'JP Morgan', 'symbol': 'JPM'},
  {'name': 'Spotify', 'symbol': 'SPOT'},
  {'name': 'Disney', 'symbol': 'DIS'},
  {'name': 'Tesla', 'symbol': 'TSLA'},
  {'name': 'Intel', 'symbol': 'INTC'}
]

Conseguimos el precio de cada una, y lo guardamos en un array

In [6]:
results = []

In [7]:
for stock in stocks:
  data = api.quote(stock['symbol'])
  result = {k: data[k] for k in ('c', 'h', 'l', 'o', 'pc', 't')}
  result = {**stock, **result}
  results.append(result)
  print(f"{stock['name']}: ${data['c']:,.2f}")

Apple: $172.03
Google: $151.08
Coca Cola: $61.13
Microsoft: $421.25
Amazon: $180.56
JP Morgan: $200.56
Spotify: $263.95
Disney: $122.45
Tesla: $175.58
Intel: $44.08


Convertir datos a DF

In [8]:
df = pd.DataFrame(results)
df.rename(columns={'name': 'company', 'c': 'current_price', 'h': 'high_price', 'l': 'low_price', 'o': 'open_price', 'pc': 'last_close_price', 't': 'price_timestamp_unix'}, inplace=True)

Categorizar columnas

In [9]:
df.company = df.company.astype(str)
df.symbol = df.symbol.astype(str)
df.dtypes

company                  object
symbol                   object
current_price           float64
high_price              float64
low_price               float64
open_price              float64
last_close_price        float64
price_timestamp_unix      int64
dtype: object

Agregamos un formato fecha legible

In [10]:
df['price_timestamp'] = pd.to_datetime(df.price_timestamp_unix, unit='s')

## Resultado Final

In [11]:
df[['company', 'symbol', 'current_price', 'high_price', 'low_price', 'open_price', 'last_close_price', 'price_timestamp']]

,company,symbol,current_price,high_price,low_price,open_price,last_close_price,price_timestamp
0,Apple,AAPL,172.0350,172.230,170.510,171.700,173.31,2024-03-28 19:38:02
1,Google,GOOGL,151.0800,151.430,150.170,150.840,150.87,2024-03-28 19:38:19
2,Coca Cola,KO,61.1350,61.260,60.995,61.140,61.03,2024-03-28 19:37:44
3,Microsoft,MSFT,421.2500,421.760,419.140,421.050,421.43,2024-03-28 19:38:14
4,Amazon,AMZN,180.5600,181.700,179.260,180.130,179.83,2024-03-28 19:37:46
5,JP Morgan,JPM,200.5600,200.710,198.580,199.490,199.52,2024-03-28 19:38:16
6,Spotify,SPOT,263.9500,268.545,258.000,258.530,260.20,2024-03-28 19:37:39
7,Disney,DIS,122.4499,123.740,121.200,121.250,120.98,2024-03-28 19:38:07
8,Tesla,TSLA,175.5800,179.570,175.490,177.459,179.83,2024-03-28 19:38:01
9,Intel,INTC,44.0750,44.600,43.710,43.797,43.77,2024-03-28 19:37:55
